In [1]:
import random
import copy
import numpy as np
import theano as T
import theano.tensor as tt
import pymc3 as pm
import pandas as pd
import rpy2
from rpy2.robjects import pandas2ri
pandas2ri.activate()
%load_ext rpy2.ipython




/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

%%R

library(ggplot2)
library(reshape)
library(grid)
library(dplyr)
library(gridExtra)
library(lme4)


paper_theme_2 <- theme_light() + theme(axis.title.x = element_text(size=18),
  axis.text.x=element_text(colour="black", 
                           size = 14), 
  axis.title.y = element_text(size = 18, vjust = 1),
  axis.text.y  = element_text(size = 14, colour="black"),
 title =element_text(size=16, face='bold'))



data_resp_loc <- read.csv("data/discrimination_dot_gaze.csv", sep="\t")
data_tracker <- read.csv("data/discrimination_tracker_data.csv", sep="\t")
data_tracker$trial_id <- floor((data_tracker$trial_id - 1)/2)
data_resp_loc$trial_id <- data_resp_loc$trial_id - 1


data_resp_loc <- data_resp_loc %>%
                filter(belowX >= 0) #%>%
                #mutate(cond=paste(round(Time1),round(Time2),sep="_"))

trials <- (unique(data_resp_loc$trial_id))
data_tracker <- data_tracker %>%
                group_by(trial_id) %>%
                filter(trial_id %in% trials)

data_resp_loc$id <- seq.int(1, nrow(data_resp_loc))
data_resp_loc$Score <- as.numeric(data_resp_loc$Score) - 1

#data_resp_loc$cond <- plyr::mapvalues(data_resp_loc$cond, 
                  #                from = c("0_0", "0_1", "1_0", "1_1"),
                        #          to = c("Short_Short", "Short_Long", "Long_Short", "Long_Long"))

data_resp_loc_old <- cbind(data_resp_loc)


/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Need help getting started? Try the cookbook for R:
http://www.cookbook-r.com/Graphs/

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: 
Attaching package: ‘dplyr’


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following object is masked from ‘package:reshape’:

    rename


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:stats’:

    filter, lag


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.5/dist-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/pyth

In [23]:
%%R -o data_resp_avs

data_resp_loc <- cbind(data_resp_loc_old)
        
data_resp_avs <- data_resp_loc %>%
                
                filter((gazeDist > 0))  %>%
                  group_by(trial_id, which_array) %>%
                  mutate(arr1 = (which_array==1)*1) %>%
                  mutate(arr2 = (which_array==2)*1) %>%
                  mutate(belowX1 = mean(belowX) * arr1) %>%
                  mutate(belowX2 = mean(belowX) * arr2)%>%
                  mutate(pathLength1 = mean(pathLength) * arr1) %>%
                  mutate(pathLength2 = mean(pathLength) * arr2)%>%
                  mutate(medFix1 = mean(medFix) * arr1) %>%
                  mutate(medFix2 = mean(medFix) * arr2) %>%
                  mutate(pctArea1 = mean(pctArea) * arr1) %>%
                  mutate(pctArea2 = mean(pctArea) * arr2) %>%
                  #mutate(within_x1 =  1/(1 + exp(-0.5 *belowX))) %>%
                  #mutate(within_x2 =   1/(1 + exp(-0.5 *belowX)))  %>%
  
                    mutate(within_x1 = 2/(1 + exp(-1 -0.25*belowX)) - 1) %>%
                      mutate(within_x2 = 2/(1 + exp(-1 -0.25*belowX)) - 1) %>%
                  # mutate(within_x1= arr1 * mean(pInGaze**0.2)) %>%
                  # mutate(within_x2= arr2 * mean(pInGaze**0.2)) %>%

                  mutate(within_x1 = arr1* sum(within_x1)) %>%
                  mutate(within_x2 = arr2*sum(within_x2)) %>%
                  top_n(n=1, wt=id)%>%
                  group_by(pid, Subject, Trial, Time1, Time2) %>%
                  mutate(pctArea1 = max(pctArea1)) %>%
                  mutate(pctArea2 = max(pctArea2)) %>%
                  mutate(belowX1 = max(belowX1)) %>%
                  mutate(belowX2 = max(belowX2)) %>%
                  mutate(within_x1 = max(within_x1)) %>%
                  mutate(within_x2=max(within_x2)) %>%
                  mutate(pathLength1 = max(pathLength1)) %>%
                  mutate(pathLength2 = max(pathLength2))%>%
                  mutate(medFix1 = max(medFix1)) %>%
                  mutate(medFix2 = max(medFix2))%>%
                 top_n(n=1, wt=id) %>%
                mutate(within_x1_pct = within_x1 / Dots_Shown1) %>%
                mutate(within_x2_pct = within_x2 / Dots_Shown2) %>% 
                mutate(within_x_pct_ratio = (within_x1 - within_x2)/((within_x1**2 + within_x2**2)**0.5)) %>%
               mutate(cond=paste(round(Time1),round(Time2),sep="_")) %>%

                mutate(log_Dots_Diff =  log(Dots_Shown1) - log(Dots_Shown2)) %>%
                mutate(Dots_Ratio = (Dots_Shown1 - Dots_Shown2)/(Dots_Shown1**2 + Dots_Shown2**2)**0.5) %>%
                mutate(Abs_Dots_Ratio = abs(Dots_Ratio))  %>%
                mutate(Dots_Counted = Dots_Counted - 1) %>%
                mutate(Score = Score - 1)



In [24]:
data_resp = pandas2ri.ri2py(data_resp_avs)

In [31]:

def remove_n(ls,n):
    ret = []
    for l in ls:
        l_rem = l[:n]
        ret.append(l_rem)
    return ret



#subj_group = data_resp.groupby(["pid", "Time1", "Time2"])
subj_group = data_resp.groupby(["pid","Subject"])
#subj_group = data_resp.groupby([])

dots_shown1,dots_shown2,dots_counted,pct_area1,pct_area2,seen1,seen2,times1,times2,pid, score=[],[],[],[],[],[],[],[],[],[],[]
for i,subj in subj_group:
    times1.append([float(i) for i in list(subj["Time1"])])
    times2.append([float(i) for i in list(subj["Time2"])])
    score.append(list(subj["Score"]))

    dots_counted.append(list(subj["Dots_Counted"]*1.))
    dots_shown1.append(list(subj["Dots_Shown1"]*1.))
    pct_area1.append(list(subj["pctArea1"]))
    seen1.append(list(subj["within_x1"]))
    #seen1.append(list(subj["within_x1_tot"]))
    dots_shown2.append(list(subj["Dots_Shown2"]*1.))
    pct_area2.append(list(subj["pctArea2"]))
    #seen2.append(list(subj["within_x2_tot"]))
    seen2.append(list(subj["within_x2"]))
    pid.append(list(subj["Subject"]))

ns_1 = [len(n) for n in seen1]
min_n1 = min(ns_1)
ns_2 = [len(n) for n in seen2]
min_n2 = min(ns_2)
min_n = min(min_n1, min_n2)


dots_counted = remove_n(dots_counted, min_n)
dots_shown1 = remove_n(dots_shown1, min_n)
pct_area1 = remove_n(pct_area1, min_n)
seen1 = remove_n(seen1, min_n)
dots_shown2 = remove_n(dots_shown2, min_n)
pct_area2 = remove_n(pct_area2, min_n)
seen2 = remove_n(seen2, min_n)
times1 = remove_n(times1, min_n)
times2 = remove_n(times2, min_n)
pid = remove_n(pid, min_n)
score=remove_n(score, min_n)

dots_counted=np.array(dots_counted)
dots_shown1=np.array(dots_shown1)
pct_area1=np.array(pct_area1)
seen1=np.array(seen1)
unseen1 = dots_shown1 - seen1
dots_shown2=np.array(dots_shown2)
pct_area2=np.array(pct_area2)
seen2=np.array(seen2)
unseen2 = dots_shown2 - seen2
score=np.array(score)
dims = (len(seen1),len(seen1[0]))

with pm.Model() as mod:

    #intercepts
    slope_grp_mu = pm.Normal("slope_grp_mu",mu=0,sd=5,testval=0.0, shape=1)
    slope_grp_sd = pm.HalfNormal("slope_grp_sd",sd=5,testval=0.25, shape=1)
    weber_grp_mu = pm.Normal("weber_grp_mu",mu=0,sd=5,testval=0.0, shape=1)
    weber_grp_sd = pm.HalfNormal("weber_grp_sd",sd=5,testval=1., shape=1)
    
    #time effects
    slope_time_grp_mu = pm.Normal("slope_time_grp_mu",mu=0,sd=5,testval=0.0, shape=1)
    slope_time_grp_sd = pm.HalfNormal("slope_time_grp_sd",sd=5,testval=0.25, shape=1)
    weber_time_grp_mu = pm.Normal("weber_time_grp_mu",mu=0,sd=5,testval=0.0, shape=1)
    weber_time_grp_sd = pm.HalfNormal("weber_time_grp_sd",sd=5,testval=1., shape=1)

    #seen effects
    #slope_seen_grp_mu = pm.Normal("slope_seen_grp_mu",mu=0,sd=5,testval=0.0, shape=1)
    #slope_seen_grp_sd = pm.HalfNormal("slope_seen_grp_sd",sd=5, testval=0.25, shape=1)

    #weber_seen_grp_mu = pm.Normal("weber_seen_grp_mu",mu=0,sd=5,testval=0.0, shape=1)
    #weber_seen_grp_sd = pm.HalfNormal("weber_seen_grp_sd",sd=5,testval=0.25, shape=1)

    #individual

    slope_time = pm.Normal("slope_time",mu=slope_time_grp_mu,sd=slope_time_grp_sd,testval=0.0, shape=(len(seen1),1))
    #slope_seen = pm.Normal("slope_seen",mu=slope_seen_grp_mu,sd=slope_seen_grp_sd,testval=0.0, shape=(len(seen1),1))
    weber_time = pm.Normal("weber_time",mu=weber_time_grp_mu,sd=weber_time_grp_sd,testval=0.0, shape=(len(seen1),1))
    #weber_seen =pm.Normal("weber_seen",mu=weber_seen_grp_mu,sd=weber_seen_grp_sd,testval=0.0, shape=(len(seen1),1))
    
    slope = pm.Normal("slope",mu=slope_grp_mu,sd=slope_grp_sd,testval=0.0, shape=(len(seen1),1))
    weber = pm.Normal("weber",mu=weber_grp_mu,sd=weber_grp_sd,testval=0.0, shape=(len(seen1),1))

    ####################
    #weber_tot = tt.exp(weber)
    weber_tot =  tt.exp(weber + weber_time * tt.log(times1) + weber_time * tt.log(times2))

    #weber_tot =  tt.exp(weber + weber_time * tt.log(times1) + 
         #                   weber_time * tt.log(times2) + 
          #              weber_seen * seen1 + weber_seen * seen2)


    #slope1,slope2 = 1,1
   # slope1 = tt.exp(slope)
   # slope2 = tt.exp(slope)
    slope1 = tt.exp(slope + slope_time * tt.log(times1))
    slope2 = tt.exp(slope + slope_time * tt.log(times2))

    #slope1 = tt.exp(slope + slope_time * tt.log(times1)+ slope_seen * seen1)
    #slope2 = tt.exp(slope + slope_time * tt.log(times2)+ slope_seen * seen2)

    guess_mean1 = dots_shown1 * slope1
    guess_mean2 = dots_shown2 * slope2
    #guess_mean1 = seen1
    #guess_mean2 = seen2

    
    ps = (guess_mean2 - guess_mean1) /  (weber_tot * ((guess_mean2**2 + guess_mean1**2)**0.5))

    ps = 0.5 * (1. + tt.erf(ps/(2.**0.5))) 
    #ps = 0.5 * (1. + tt.erf(abs(ps)/(2.**0.5))) 

    #noise_grp = pm.Beta("noise", 1,20, shape=1, testval=0.01)
    noisy_ps = ps + (0.5 - ps) * 0.1

   # noisy_ps = ps
    #likelihood = pm.Bernoulli('likelihood', noisy_ps,  
     #                         observed=score, shape=dims)

    likelihood = pm.Bernoulli('likelihood', noisy_ps,  observed=dots_counted, shape=dims)
    #likelihood = pm.Binomial('likelihood',np.ones(dims), noisy_ps,  observed=dots_counted, shape=dims)

    trace = pm.sample(1000, tune=100, cores=2,chains=2, njobs=2)

summary = pm.summary(trace)
print(summary)
    
    


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/usr/local/lib/python3.5/dist-packages/pymc3/model.py:384: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if not np.issubdtype(var.dtype, float):
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [weber, slope, weber_time, slope_time, weber_time_grp_sd_log__, weber_time_grp_mu, slope_time_grp_sd_log__, slope_time_grp_mu, weber_grp_sd_log__, weber_grp_mu, slope_grp_sd_log__, slope_grp_mu]
100%|██████████| 1100/1100 [11:10<00:00,  1.64it/s]
There were 48 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, increase target_accept or reparameterize.
There were 113 divergences after tuning. Increase `target_accept` or reparameterize.
The chain reached the maximum tree depth. Increase max_treedepth, incre

                          mean        sd  mc_error   hpd_2.5   hpd_97.5  \
slope_grp_mu__0      -2.087896  2.974495  0.281272 -7.984388   4.077815   
weber_grp_mu__0      -2.000395  0.106077  0.002742 -2.213020  -1.799805   
slope_time_grp_mu__0  0.032845  0.006740  0.000170  0.019200   0.045218   
weber_time_grp_mu__0 -0.059497  0.039083  0.001000 -0.137465   0.013342   
slope_time__0_0       0.028138  0.013947  0.000393 -0.001106   0.052971   
slope_time__1_0       0.023244  0.015268  0.000636 -0.010020   0.049022   
slope_time__2_0       0.029374  0.014022  0.000346  0.001427   0.057899   
slope_time__3_0       0.034991  0.014165  0.000340  0.004824   0.061224   
slope_time__4_0       0.032551  0.013752  0.000266  0.004861   0.060268   
slope_time__5_0       0.041050  0.016064  0.000572  0.009826   0.073104   
slope_time__6_0       0.034359  0.013186  0.000352  0.007526   0.061101   
slope_time__7_0       0.033009  0.014294  0.000377  0.004330   0.062259   
slope_time__8_0       0.0

In [6]:
summary_mean = summary["mean"]
summary_sd = summary["sd"]
summary_low = summary["hpd_2.5"]
summary_high = summary["hpd_97.5"]

vals = {}
rownames = list(summary.index)

n_groups = seen1.shape[0]
n_per_group = seen1.shape[1]

print(summary_mean)
print(summary_sd)


vals = {}
for i in range(len(rownames)):
    rowname = rownames[i]
    rowname = rowname[:rowname.find("__")]
    
    if 'grp' in rowname:
        vals[rowname] = [summary_mean[i] for _ in range(n_per_group*n_groups)]
        vals[rowname+"_sd"] = [summary_sd[i] for _ in range(n_per_group*n_groups)]
        vals[rowname+"_low"] = [summary_low[i] for _ in range(n_per_group*n_groups)]
        vals[rowname+"_high"] = [summary_high[i] for _ in range(n_per_group*n_groups)]

    else:
        if rowname not in vals:
            vals[rowname] = [summary_mean[i] for _ in range(n_per_group)]
            vals[rowname+"_sd"] = [summary_sd[i] for _ in range(n_per_group)]
            vals[rowname+"_low"] = [summary_low[i] for _ in range(n_per_group)]
            vals[rowname+"_high"] = [summary_high[i] for _ in range(n_per_group)]
        else:
            vals[rowname] += [summary_mean[i] for _ in range(n_per_group)]
            vals[rowname+"_sd"] += [summary_sd[i] for _ in range(n_per_group)]
            vals[rowname+"_low"] += [summary_low[i] for _ in range(n_per_group)]
            vals[rowname+"_high"] += [summary_high[i] for _ in range(n_per_group)]

vals["pid"] = np.ravel(pid)
vals["Time1"] = np.ravel(times1)
vals["seen1"] = np.ravel(seen1)
vals["Time2"] = np.ravel(times2)
vals["seen2"] = np.ravel(seen2)
vals["Dots_Shown1"] = np.ravel(dots_shown1)
vals["Dots_Shown2"] = np.ravel(dots_shown2)
vals["Dots_Counted"] = np.ravel(dots_counted)
sum_vals=pd.DataFrame(vals)
pandas2ri.py2ri(sum_vals)


weber_grp_mu__0   -1.744148
weber__0_0        -1.743783
weber__1_0        -1.757067
weber__2_0        -1.757984
weber__3_0        -1.756022
weber__4_0        -1.720913
weber__5_0        -1.711168
weber__6_0        -1.720712
weber__7_0        -1.730736
weber__8_0        -1.757905
weber__9_0        -1.719508
weber__10_0       -1.774115
weber__11_0       -1.785440
weber__12_0       -1.801529
weber__13_0       -1.782628
weber__14_0       -1.746801
weber__15_0       -1.689742
weber__16_0       -1.744254
weber__17_0       -1.740231
weber__18_0       -1.767208
weber__19_0       -1.746478
weber__20_0       -1.770591
weber__21_0       -1.747548
weber__22_0       -1.760013
weber__23_0       -1.762308
weber__24_0       -1.711990
weber__25_0       -1.689527
weber_grp_sd__0    0.101644
Name: mean, dtype: float64
weber_grp_mu__0    0.061204
weber__0_0         0.110634
weber__1_0         0.107897
weber__2_0         0.107671
weber__3_0         0.118621
weber__4_0         0.126127
weber__5_0         0.

Dots_Counted,Dots_Shown1,Dots_Shown2,...,weber_high,weber_low,weber_sd
1.000000,55.000000,50.000000,...,-1.511447,-1.946175,0.110634
0.000000,61.000000,43.000000,...,-1.511447,-1.946175,0.110634
1.000000,50.000000,54.000000,...,-1.511447,-1.946175,0.110634
1.000000,55.000000,56.000000,...,-1.511447,-1.946175,0.110634
...,...,...,...,...,...,...
0.000000,50.000000,51.000000,...,-1.425225,-1.912695,0.124437
0.000000,25.000000,19.000000,...,-1.425225,-1.912695,0.124437


In [7]:
%%R -i sum_vals summary

sum_vals <- sum_vals %>%
                mutate(ratio=(Dots_Shown1-Dots_Shown2)/(weber*(Dots_Shown1**2 + Dots_Shown2**2)**0.5))



print(mean(sum_vals$slope_grp_mu))
print(mean(sum_vals$slope_time_grp_mu))
print(mean(sum_vals$slope_seen_grp_mu))
print(mean(sum_vals$weber_grp_mu))
print(mean(sum_vals$weber_time_grp_mu))
print("")
print(mean(sum_vals$weber_seen_grp_mu))
print(mean(sum_vals$weber_seen_grp_mu_high))
print(mean(sum_vals$weber_seen_grp_mu_low))



[1] NA
[1] NA
[1] NA
[1] -1.744148
[1] NA
[1] ""
[1] NA
[1] NA
[1] NA
